### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
18,43273962,79085,89481,1001,3,79,1159,6,2025-10-03 21:59,2025-10-03 21:59,0,1,15495,4,1
19,43092752,79085,48378,2075,3,70,1160,1,2025-10-05 21:59,2025-10-05 21:59,0,5,4952,1,1
20,43092762,121325,79085,2075,4,70,1160,4,2025-10-08 21:59,2025-10-08 21:59,0,4,3055,2,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

3

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,79085,46282,34388,0,3,5,0,0,5-4-1a,6,7,9,43273962
1,89481,23669,35563,1,15,1,1,0,5-2-1-2,10,10,7,43273962
0,79085,44532,31863,0,2,1,0,0,3-4-3a,6,7,8,43092752
1,48378,25425,38094,0,11,0,0,0,424***,9,10,7,43092752
0,121325,52311,27161,0,4,1,1,0,Normal,4,3,2,43092762
1,79085,18165,43315,0,25,0,0,0,3-4-3aQJ,7,7,7,43092762


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1966

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,89569,4,9,3,0,1,11,6,9505011003004,1349
1,6276,4,8,2,2,0,10,6,8504010002010,1349
2,78671,4,7,2,1,1,17,7,7510017002005,1349
3,90105,4,7,2,1,1,3,3,7500003002011,1349
4,79587,4,6,2,0,2,11,12,6499011002008,1349
5,142689,4,6,2,0,2,6,7,6499006002002,1349
6,124139,4,5,1,2,1,12,3,5509012001009,1349
7,78606,4,5,1,2,1,8,5,5503008001012,1349
8,78751,4,5,1,2,1,10,10,5500010001007,1349
9,23755,4,5,1,2,1,6,6,5500006001006,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()